In [1]:
# 0 define backend

%env DDE_BACKEND=tensorflow

env: DDE_BACKEND=tensorflow


In [2]:
# 2 define xla

%env XLA_FLAGS=--xla_gpu_cuda_data_dir=/usr/lib/cuda

# below does NOT work, dde won't parse it
# %env TF_XLA_FLAGS=--xla_gpu_cuda_data_dir=/usr/lib/cuda

# using /usr/lib/nvidia-cuda-toolkit does NOT work

env: XLA_FLAGS=--xla_gpu_cuda_data_dir=/usr/lib/cuda


In [3]:
%env XLA_FLAGS

'--xla_gpu_cuda_data_dir=/usr/lib/cuda'

In [ ]:
# %env TF_XLA_FLAGS="--tf_xla_auto_jit=1 --tf_xla_cpu_global_jit"

# =1, =2, =-1 both are slow still

In [4]:
# 1 import stuff

import os, json

# "tensorflow.compat.v1", "tensorflow", "pytorch"
def set_default_backend(backend_name):
    default_dir = os.path.join(os.path.expanduser("~"), ".deepxde")
    if not os.path.exists(default_dir):
        os.makedirs(default_dir)
    config_path = os.path.join(default_dir, "config.json")
    with open(config_path, "w") as config_file:
        json.dump({"backend": backend_name.lower()}, config_file)
    print(
        'Setting the default backend to "{}". You can change it in the '
        "~/.deepxde/config.json file or export the DDE_BACKEND environment variable. "
        "Valid options are: tensorflow.compat.v1, tensorflow, pytorch, jax, paddle (all lowercase)".format(
            backend_name
        )
    )
    
import tensorflow as tf
set_default_backend("tensorflow")

# import tensorflow.compat.v1 as tf
# backend = "tensorflow_v1"
# set_default_backend("tensorflow.compat.v1")

# tf.config.run_functions_eagerly(False)
# tf.disable_eager_execution()
# tf.enable_eager_execution()
# tf.compat.v1.disable_eager_execution()

# eager execution for tensorflow backend works, but is VERY slow
# disabling it leads to EagerFunction error

# import torch
# backend = "pytorch"
# set_default_backend("pytorch")

import deepxde as dde

dde.model.backend_name = "tensorflow"

dde.config.enable_xla_jit(True)
# this is key for speed in v2?
# happens by default for tensorflow v1?

# dde.config.disable_xla_jit()
# if you disable, you will run out of memory

import numpy as np
from scipy import stats
import numpy as np
import matplotlib.pyplot as plt
import pylab
from numpy import linalg as LA
import math
import scipy.io
from os.path import dirname, join as pjoin
from scipy.stats import truncnorm, norm

import matplotlib
matplotlib.use("TkAgg")

import matplotlib.pyplot as plt
params = {'backend': 'ps',
          'xtick.labelsize': 12,
          'ytick.labelsize': 12,
          'legend.handlelength': 1,
          'legend.borderaxespad': 0,
          'font.family': 'serif',
          'font.serif': ['Computer Modern Roman'],
          'ps.usedistiller': 'xpdf',
          'text.usetex': True,
          # include here any neede package for latex
          'text.latex.preamble': [r'\usepackage{amsmath}'],
          }
plt.rcParams.update(params)
plt.style.use('seaborn-white')

import scipy.integrate as integrate

import time

import shutil

from scipy.interpolate import griddata as gd

gpu_devices = tf.config.experimental.list_physical_devices('GPU')
for device in gpu_devices:
    print(device)
    tf.config.experimental.set_memory_growth(device, True)

# os.environ['XLA_FLAGS'] = "--xla_gpu_cuda_data_dir=/usr/lib/cuda"
print(dde.model.backend_name)

2022-08-30 09:08:19.056465: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
Using backend: tensorflow



Setting the default backend to "tensorflow". You can change it in the ~/.deepxde/config.json file or export the DDE_BACKEND environment variable. Valid options are: tensorflow.compat.v1, tensorflow, pytorch, jax, paddle (all lowercase)
Enable just-in-time compilation with XLA.



2022-08-30 09:08:20.116079: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-30 09:08:20.126848: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-30 09:08:20.127260: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


Enable just-in-time compilation with XLA.

PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')
tensorflow


In [ ]:
%env XLA_FLAGS

In [ ]:
%env PATH

In [5]:
# 3 define training parameters

state_min = -2.5
state_max = 2.5
# shrinking this helps a lot

######################################

N = 2000

T_t = 5.0

epsilon=.001

# j1, j2, j3=3,2,1
j1, j2, j3 =1,1,2 # axis-symmetric case

######################################

# q=1 # state cost

q_statepenalty_gain = 0 # 0.5

######################################

mu_0 = 2.0
sigma_0 = 1.0

mu_T = 0.0
sigma_T = 1.0

######################################

samples_between_initial_and_final = 12000 # 10^4 order, 20k = out of memory
initial_and_final_samples = 1000 # some 10^3 order

######################################

num_epochs = 20000

id_prefix = "run_%s_%.3f_%.3f__%.3f__%d__%.3f__%d_%d_%d__%.3f__%.3f_%.3f__%.3f_%.3f__%d_%d__%d" % (
    dde.model.backend_name,
    state_min,
    state_max,
    T_t,
    N,
    epsilon,
    j1, j2, j3,
    q_statepenalty_gain,
    mu_0, sigma_0,
    mu_T, sigma_T,
    samples_between_initial_and_final, initial_and_final_samples,
    num_epochs
)

print(T_t)
print(id_prefix)

# id_prefix is a id for this training

5.0
run_tensorflow_-2.500_2.500__5.000__2000__0.001__1_1_2__0.000__2.000_1.000__0.000_1.000__12000_1000__20000


In [6]:
# 4 pde

def pde(x, y):
    """Euler system.
    dy1_t = g(x)-1/2||Dy1_x||^2-<Dy1_x,f>-epsilon*Dy1_xx
    dy2_t = -D.(y2*(f)+Dy1_x)+epsilon*Dy2_xx
    All collocation-based residuals are defined here
    """
    y1, y2 = y[:, 0:1], y[:, 1:]
    dy1_x = tf.gradients(y1, x)[0]
    dy1_x, dy1_y, dy1_z, dy1_t = dy1_x[:,0:1], dy1_x[:,1:2], dy1_x[:,2:3], dy1_x[:,3:]
    
    dy1_xx = tf.gradients(dy1_x, x)[0][:, 0:1]
    dy1_yy = tf.gradients(dy1_y, x)[0][:, 1:2]
    dy1_zz = tf.gradients(dy1_z, x)[0][:, 2:3] 
    
    dy2_x = tf.gradients(y2, x)[0]
    dy2_x, dy2_y, dy2_z, dy2_t = dy2_x[:,0:1], dy2_x[:,1:2], dy2_x[:,2:3], dy2_x[:,3:]    
    
    dy2_xx = tf.gradients(dy2_x, x)[0][:, 0:1]
    dy2_yy = tf.gradients(dy2_y, x)[0][:, 1:2]
    dy2_zz = tf.gradients(dy2_z, x)[0][:, 2:3]     

    f1=x[:, 1:2]*x[:, 2:3]*(j2-j3)/j1
    f2=x[:, 0:1]*x[:, 2:3]*(j3-j1)/j2
    f3=x[:, 0:1]*x[:, 1:2]*(j1-j2)/j3
    
    d_f1dy1_y2_x=tf.gradients((f1+dy1_x)*y2,x)[0][:, 0:1]
    d_f2dy1_y2_y=tf.gradients((f2+dy1_y)*y2,x)[0][:, 1:2]
    d_f3dy1_y2_z=tf.gradients((f3+dy1_x)*y2,x)[0][:, 2:3]

    # stay close to origin while searching, penalizes large state distance solutions
    q = q_statepenalty_gain*(x[:, 0:1] * x[:, 0:1] + x[:, 1:2] * x[:, 1:2] + x[:, 2:3] * x[:, 2:3])
    
    # also try
    # q = 0 # minimum effort control
    
    # TODO: verify this expression
    return [
        -dy1_t+q-.5*(dy1_x*dy1_x+dy1_y*dy1_y+dy1_z*dy1_z)-dy1_x*f1-dy1_y*f2-dy1_z*f3-epsilon*(dy1_xx+dy1_yy+dy1_zz),
        -dy2_t-(d_f1dy1_y2_x+d_f2dy1_y2_y+d_f3dy1_y2_z)+epsilon*(dy2_xx+dy2_yy+dy2_zz),
    ]

In [7]:
# 5 more pde, boundary

def boundary(_, on_initial):
    return on_initial

def pdf1d_0(x,y,z):
    '''
    a, b = (state_min - mu) / sigma, (state_max - mu) / sigma # must match sampling
    rho_x=truncnorm.pdf(x, a, b, loc = mu, scale = sigma)
    rho_y=truncnorm.pdf(y, a, b, loc = mu, scale = sigma)
    rho_z=truncnorm.pdf(z, a, b, loc = mu, scale = sigma) # replace with np.normal? gaussian
    '''

    rho_x = norm.pdf(x, mu_0, sigma_0)
    rho_y = norm.pdf(y, mu_0, sigma_0)
    rho_z = norm.pdf(z, mu_0, sigma_0)
    
#     rho_x = tf.random.normal(
#         [1],
#         mean=x,
#         stddev=mu_0,
#         dtype=tf.dtypes.float16,
#         seed=None,
#         name=None
#     )
#     rho_y = tf.random.normal(
#         [1],
#         mean=y,
#         stddev=mu_0,
#         dtype=tf.dtypes.float16,
#         seed=None,
#         name=None
#     )
#     rho_z = tf.random.normal(
#         shape,
#         mean=z,
#         stddev=mu_0,
#         dtype=tf.dtypes.float16,
#         seed=None,
#         name=None
#     )

    return rho_x*rho_y*rho_z

def pdf1d_T(x,y,z):
    '''
    a, b = (state_min - mu) / sigma, (state_max - mu) / sigma
    rho_x=truncnorm.pdf(x, a, b, loc = mu, scale = sigma)
    rho_y=truncnorm.pdf(y, a, b, loc = mu, scale = sigma)
    rho_z=truncnorm.pdf(z, a, b, loc = mu, scale = sigma)
    '''

    rho_x = norm.pdf(x, mu_T, sigma_T)
    rho_y = norm.pdf(y, mu_T, sigma_T)
    rho_z = norm.pdf(z, mu_T, sigma_T)
    
    return rho_x*rho_y*rho_z

In [8]:
# 6 define data and net

x_T = np.transpose(np.linspace(state_min, state_max, N))
y_T = np.transpose(np.linspace(state_min, state_max, N))
z_T = np.transpose(np.linspace(state_min, state_max, N))

x_T=x_T.reshape(len(x_T),1)
y_T=y_T.reshape(len(y_T),1)
z_T=z_T.reshape(len(z_T),1)

time=T_t*np.ones((len(x_T), 1))

terminal_time=np.hstack((x_T,y_T,z_T,time))

rho_T=pdf1d_T(x_T,y_T,z_T).reshape(len(x_T),1)

rho_T_BC = dde.icbc.PointSetBC(terminal_time, rho_T, component=1)

geom=dde.geometry.geometry_3d.Cuboid(
    [state_min, state_min, state_min],
    [state_max, state_max, state_max])
timedomain = dde.geometry.TimeDomain(0., T_t)
geomtime = dde.geometry.GeometryXTime(geom, timedomain)

rho_0_BC = dde.icbc.IC(
    geomtime,
    lambda x: pdf1d_0(x[:,0:1],x[:,1:2],x[:,2:3]),
    boundary,
    component=1)

data = dde.data.TimePDE(
    geomtime,
    pde, 
    [rho_0_BC,rho_T_BC],
    num_domain=samples_between_initial_and_final,
    num_initial=initial_and_final_samples)

net = dde.nn.FNN([4] + [70] *3  + [2], "tanh", "Glorot normal")

2022-08-30 09:08:28.307004: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
/usr/local/home/cyan3/miniforge/envs/tf/lib/python3.9/site-packages/skopt/sampler/sobol.py:246: UserWarning: The balance properties of Sobol' points require n to be a power of 2. 0 points have been previously generated, then: n=0+12002=12002. 
  warnings.warn("The balance properties of Sobol' points require "
2022-08-30 09:08:28.308267: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-30 09:08:28.308616: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA nod

In [9]:
# 7 define model

model = dde.Model(data, net)

print(model)

In [10]:
# 8 make a directory to save data

dirname = './%s' % (id_prefix)
print(dirname)

if not os.path.exists(dirname):
    print("making dirname")
    os.mkdir(dirname)
else:
    print("dir exists")
    for file in os.listdir(dirname):
        print(file)

./run_tensorflow_-2.500_2.500__5.000__2000__0.001__1_1_2__0.000__2.000_1.000__0.000_1.000__12000_1000__20000
dir exists
checkpoint
run_tensorflow_-2.500_2.500__5.000__2000__0.001__1_1_2__0.000__2.000_1.000__0.000_1.000__12000_1000__20000_loss.dat
run_tensorflow_-2.500_2.500__5.000__2000__0.001__1_1_2__0.000__2.000_1.000__0.000_1.000__12000_1000__20000_train.dat
run_tensorflow_-2.500_2.500__5.000__2000__0.001__1_1_2__0.000__2.000_1.000__0.000_1.000__12000_1000__20000_test.dat
run_tensorflow_-2.500_2.500__5.000__2000__0.001__1_1_2__0.000__2.000_1.000__0.000_1.000__12000_1000__20000.ipynb
run_tensorflow_-2.500_2.500__5.000__2000__0.001__1_1_2__0.000__2.000_1.000__0.000_1.000__12000_1000__20000-149.ckpt.data-00000-of-00001
run_tensorflow_-2.500_2.500__5.000__2000__0.001__1_1_2__0.000__2.000_1.000__0.000_1.000__12000_1000__20000-149.ckpt.index
loss.png
vopt_xt.dat
run_tensorflow_-2.500_2.500__5.000__2000__0.001__1_1_2__0.000__2.000_1.000__0.000_1.000__12000_1000__20000_test.mat
run_tensorfl

In [11]:
# 9 stop as soon as loss is low enough

ck_path = "%s/%s/model" % (os.path.abspath("./"), id_prefix)

class EarlyStoppingFixed(dde.callbacks.EarlyStopping):
    def on_epoch_end(self):
        current = self.get_monitor_value()
        if self.monitor_op(current - self.min_delta, self.best):
            self.best = current
            # must meet baseline first
            self.wait += 1
            if self.wait >= self.patience:
                self.stopped_epoch = self.model.train_state.epoch
                self.model.stop_training = True
        else:
            self.wait = 0
                
    def on_train_end(self):
        if self.stopped_epoch > 0:
            print("Epoch {}: early stopping".format(self.stopped_epoch))
        
        self.model.save(ck_path, verbose=True)

earlystop_cb = EarlyStoppingFixed(baseline=1e-3, patience=0)
# no patience otherwise keeps going to improve but bounces around

# modelcheckpt_cb = dde.callbacks.ModelCheckpoint(ck_path, verbose=True, save_better_only=True, period=1000)

# model.compile("adam", lr=1e-3,external_trainable_variables=[])
# losshistory, train_state = model.train(epochs=num_epochs, callbacks=[])



In [12]:
# 10 compile

model.compile(optimizer="adam", lr=1e-3)

Compiling model...
adam
['LBFGS_options', '__all__', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', '_load_backend', 'backend_name', 'config', 'get', 'importlib', 'is_external_optimizer', 'optimizers', 'set_LBFGS_options', 'sys', 'tensorflow', 'tfp_optimizer']
'compile' took 0.000579 s



In [13]:
# 11 compile and train model

losshistory, train_state = model.train(
    display_every=1,
    iterations=num_epochs,
    callbacks=[earlystop_cb])

# variable = dde.callbacks.VariableValue([S], period=600, filename="variables.dat")
# losshistory, train_state = model.train(epochs=20000, callbacks=[variable])

Training model...

Cause: could not parse the source code of <function <lambda> at 0x7fd0decfcd30>: no matching AST found among candidates:
# coding=utf-8
lambda x, on: np.array([on_boundary(x[i], on[i]) for i in range(len(x))])
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: could not parse the source code of <function <lambda> at 0x7fd0decfcd30>: no matching AST found among candidates:
# coding=utf-8
lambda x, on: np.array([on_boundary(x[i], on[i]) for i in range(len(x))])
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


2022-08-30 09:08:46.841066: I tensorflow/compiler/xla/service/service.cc:170] XLA service 0x5564935d6ba0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2022-08-30 09:08:46.841088: I tensorflow/compiler/xla/service/service.cc:178]   StreamExecutor device (0): NVIDIA RTX A2000 Laptop GPU, Compute Capability 8.6
2022-08-30 09:08:46.872862: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:263] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2022-08-30 09:08:47.476763: W tensorflow/stream_executor/gpu/asm_compiler.cc:111] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.
2022-08-30 09:08:47.477985: W tensorflow/stream_executor/gpu/asm_compiler.cc:230] Falling back to the CUDA 

Step      Train loss                                  Test loss                                   Test metric
0         [5.11e-02, 5.89e-02, 8.78e-02, 6.08e-02]    [5.11e-02, 5.89e-02, 8.78e-02, 6.08e-02]    []  


2022-08-30 09:08:50.984047: W tensorflow/stream_executor/gpu/asm_compiler.cc:111] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.
2022-08-30 09:09:01.786099: E tensorflow/compiler/xla/service/slow_operation_alarm.cc:61] Constant folding an instrution is taking > 1s:

  %dot.51 = f32[128000,70]{1,0} dot(f32[128000,70]{1,0} %constant.263, f32[70,70]{1,0} %constant.395), lhs_contracting_dims={1}, rhs_contracting_dims={1}

This isn't necessarily a bug; constant-folding is inherently a trade-off between compilation time and speed at runtime.  XLA has some guards that attempt to keep constant folding from taking too long, but fundamentally you'll always be able to come up with an input program that takes a long time.

If you'd like to file a bug, run with envvar XLA_FLA

1         [1.90e-02, 1.86e-02, 3.06e-02, 7.21e-03]    [1.90e-02, 1.86e-02, 3.06e-02, 7.21e-03]    []  


2022-08-30 09:10:19.722424: W tensorflow/stream_executor/gpu/asm_compiler.cc:111] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.
2022-08-30 09:11:43.776808: W tensorflow/stream_executor/gpu/asm_compiler.cc:111] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.


KeyboardInterrupt: 

In [ ]:
# 12 save training, test, loss

dde.saveplot(losshistory, train_state, issave=True, isplot=False)
# test.dat will have 2*initial_and_final_samples + samples_between_initial_and_final + N (terminal_time)?

In [ ]:
%%javascript
IPython.notebook.kernel.execute('nb_name = "' + IPython.notebook.notebook_name + '"')

In [ ]:
# 13 move notebook, dat to folder

if os.path.exists('./loss.dat'):
    print("found")
    os.rename('./loss.dat', '%s/loss.dat' % (dirname))
    os.rename('./train.dat', '%s/train.dat' % (dirname))
    os.rename('./test.dat', '%s/test.dat' % (dirname))

nb_full_path = os.path.join(os.getcwd(), nb_name)
shutil.copyfile('./%s' % (nb_name), './%s/notebook.ipynb' % (dirname))

In [ ]:
# 14 remove previous saved model

found = False
for file in os.listdir(dirname):
    if "ckpt" in file:
        print("removing %s" % (file))
        os.remove("%s/%s" % (dirname, file))
        found = True
    else:
        pass

if not found:
    print("noop")

In [ ]:
# 15 save model

model_path = model.save("%s/%s/model" % (os.path.abspath("./"), id_prefix))
print(model_path)

In [ ]:
# 16 plot loss

loss_loaded = np.genfromtxt('./%s/loss.dat' % (id_prefix))

# import ipdb; ipdb.set_trace();

# [0] epoch
# [1] y1, psi, hjb
# [2] y2, rho, plank pde
# [3] rho0, initial
# [4] rhoT, terminal

epoch = loss_loaded[:, 0]
y1_psi_hjb = loss_loaded[:, 1]
y2_rho_plankpde = loss_loaded[:, 2]
rho0_initial = loss_loaded[:, 3]
rhoT_terminal = loss_loaded[:, 4]

fig, ax = plt.subplots()
ax.set_yscale('log')

line1, = ax.plot(epoch, y1_psi_hjb, color='orange', lw=1, label='HJB PDE')
line2, = ax.plot(epoch, y2_rho_plankpde, color='blue', lw=1, label='Controlled Fokker-Planck PDE')
line3, = ax.plot(epoch, rho0_initial, color='red', lw=1, label='p0 boundary condition')
line4, = ax.plot(epoch, rhoT_terminal, color='purple', lw=1, label='pT boundary condition')

ax.grid()
ax.legend(loc="upper right")
ax.set_title('training error/residual plots: mu0=2 -> muT=0')

plot_fname = "%s/%s/loss.png" % (os.path.abspath("./"), id_prefix)
plt.savefig(plot_fname, dpi=300)
print("saved plot")

plt.show()

In [ ]:
# 17 restore model to model_restored

model_restored = dde.Model(data, net)

ckpt_filename = None
for file in os.listdir(dirname):
    if file.endswith("ckpt.index"):
        print(file)        
        ckpt_filename = file.replace(".index", "")
print("")
print("ckpt_filename")
print(ckpt_filename)
print("")
model_name = "%s/%s/%s" % (os.path.abspath("./"), id_prefix, ckpt_filename)
print(model_name)

print("")
print(model_restored)
model_restored.restore(model_name)
print(model_restored)

In [ ]:
# 18 load test data

test = np.genfromtxt('./%s/test.dat' % (id_prefix))
# test_timesorted = test[test[:, 3].argsort()]
# sort AGAIN by output because a lot of samples @ t=0, t=5
ind = np.lexsort((test[:,4],test[:,3])) # sorts by [3] (t) then by [4] (psi)
test_timesorted = test[ind]
source_t = test_timesorted[:, 3]

print(test_timesorted)

In [ ]:
# 19 save test data to mat for matlab

import scipy.io
mat = {
    "test_timesorted" : test_timesorted,
}
mat_fname = '%s/%s/test.mat' % (os.path.abspath("./"), id_prefix)
scipy.io.savemat(mat_fname, mat)
print("saved mat", mat_fname)

In [ ]:
# 20 print loaded test data, try np.gradient

print(test_timesorted[:, 4])
print(test_timesorted[:, 0])
tmp = np.gradient(test_timesorted[:, 4], test_timesorted[:, 0])
print(tmp[:10])

In [ ]:
# 21 make input tensor

inp = test_timesorted[:, 0:4] # x1, x2, x3, t
inp_tensor = tf.convert_to_tensor(inp, dtype=tf.float32)

In [ ]:
# 22 get tf gradient with model, compare to test output, save

print("")
print("inp")
print(inp, inp.shape)

######################################

with tf.GradientTape(persistent=True) as tape:
    tape.watch(inp_tensor)
    # preds = model.net(inp_tensor)

    # you can take gradient of BOTH psi / rho w.r.t. input
    # but we only care about psi model.net[:, 0]
    psi = model.net(inp_tensor)[:, 0]

######################################

# print("")
# print("preds")
# print(preds)

print("")
print("psi")
print(psi)

######################################

print("")
print("test output")
# print(test_timesorted[:, 4:6])
print(test_timesorted[:, 4])

######################################

# gradient = tape.gradient(preds, inp_tensor)
# gradient = gradient.numpy()

vopt_xt = tape.gradient(psi, inp_tensor) # nx4
vopt_xt = vopt_xt.numpy()

######################################

# gradient_fname = "%s/%s/gradient.dat" % (os.path.abspath("./"), id_prefix)
# np.savetxt(gradient_fname, gradient)
# print("")
# print("gradient")
# print(gradient, gradient.shape)
# print("saved gradient")

vopt_xt_fname = "%s/%s/vopt_xt.dat" % (os.path.abspath("./"), id_prefix)
np.savetxt(vopt_xt_fname, vopt_xt)
print("")
print("vopt_xt")
print(vopt_xt)
print("saved vopt_xt")

del tape

In [ ]:
# 23 get tf gradient with model restored from file

with tf.GradientTape() as tape:
    tape.watch(inp_tensor)

    # restored_preds = model_restored.net(inp_tensor)
    
    # you can take gradient of BOTH psi / rho w.r.t. input
    # but we only care about psi model.net[:, 0]
    restored_psi = model_restored.net(inp_tensor)[:, 0]

######################################
  
# print("")
# print("restored_preds")
# print(restored_preds)

print("")
print("restored_psi")
print(restored_psi)

######################################

print("")
print("test output")
# print(test_timesorted[:, 4:6])
print(test_timesorted[:, 4])

######################################

# restored_gradient = tape.gradient(restored_preds, inp_tensor)
# restored_gradient = restored_gradient.numpy()

restored_vopt_xt = tape.gradient(restored_psi, inp_tensor)
restored_vopt_xt = restored_vopt_xt.numpy()

######################################

print("")
print("restored_vopt_xt")
print(restored_vopt_xt)

# restored_gradient_fname = "%s/%s/restored_gradient.dat" % (os.path.abspath("./"), id_prefix)
# np.savetxt(restored_gradient_fname, restored_gradient)
# print("")
# print("restored_gradient")
# print(restored_gradient, restored_gradient.shape)
# print("saved restored_gradient")

# restored_vopt_xt_fname = "%s/%s/restored_vopt_xt.dat" % (os.path.abspath("./"), id_prefix)
# np.savetxt(restored_vopt_xt_fname, restored_vopt_xt)
# print("saved restored_vopt_xt")

In [ ]:
# 24 load matlab gradient from file

matlab_v_name = "%s/%s" % (os.path.abspath("./"), 'matlab_vopt_xt.mat')
restored_vopt_xt = scipy.io.loadmat(matlab_v_name)["vopt_xt"]
restored_vopt_xt = np.nan_to_num(restored_vopt_xt, copy=False)

print(restored_vopt_xt)

In [ ]:
# 25 shuffle rows, recompute tf gradient, and sort again and compare
# shows that gradient does NOT depend on ordering of rows

np.random.shuffle(test)
inp_shuffled = test[:, 0:4] # x1, x2, x3, t
inp_shuffled_tensor = tf.convert_to_tensor(inp_shuffled, dtype=tf.float32)
ind = np.lexsort((test[:,4],test[:,3])) # sorts by [3] (t) then by [4] (psi)
# sort AGAIN by output because a lot of samples @ t=0, t=5

with tf.GradientTape() as tape:
    tape.watch(inp_shuffled_tensor)

    # shuffled_preds = model_restored.net(inp_shuffled_tensor)

    # you can take gradient of BOTH psi / rho w.r.t. input
    # but we only care about psi model.net[:, 0]
    shuffled_psi = model_restored.net(inp_shuffled_tensor)[:, 0]

######################################
  
# print("")
# print("shuffled_preds")
# print(shuffled_preds)

print("")
print("shuffled_psi")
print(shuffled_psi)

######################################

print("")
print("test output")
# print(test[:, 4:6])
print(test[:, 4])

######################################

# shuffled_gradient = tape.gradient(shuffled_preds, inp_shuffled_tensor)
# shuffled_gradient = shuffled_gradient.numpy()

shuffled_vopt_xt = tape.gradient(shuffled_psi, inp_shuffled_tensor)
shuffled_vopt_xt = shuffled_vopt_xt.numpy()

######################################

# shuffled_gradient_sorted = shuffled_gradient[ind]
shuffled_vopt_xt_sorted = shuffled_vopt_xt[ind]

print("")
# print("shuffled_gradient_sorted")
# print(shuffled_gradient_sorted, shuffled_gradient_sorted.shape)
print("shuffled_vopt_xt_sorted")
print(shuffled_vopt_xt_sorted, shuffled_vopt_xt_sorted.shape)

# if this matches restored_gradient and gradient, that means this gradient is order invariant and is 'real'

In [ ]:
# 26 make interp source grids, save

vinterp_N = 50
vinterp_T = 50

x_1_ = np.linspace(state_min, state_max, vinterp_N)
x_2_ = np.linspace(state_min, state_max, vinterp_N)
x_3_ = np.linspace(state_min, state_max, vinterp_N)
t_ = np.linspace(0, T_t, vinterp_T)

end_grid_x1, end_grid_x2, end_grid_x3 = np.meshgrid(
  x_1_,
  x_2_,
  x_3_, copy=False)

mid_grid_x1, mid_grid_x2, mid_grid_x3, mid_grid_t = np.meshgrid(
  x_1_,
  x_2_,
  x_3_,
  t_, copy=False)

save = False
if save:
    x1_x2_x3_t = {
        "x_1_" : x_1_,
        "x_2_" : x_2_,
        "x_3_" : x_3_,
        "t_" : t_,
    }
    mat_fname = '%s/%s/post_predict_x1_x2_x3_t.mat' % (
        os.path.abspath("./"),
        id_prefix)
    scipy.io.savemat(mat_fname, x1_x2_x3_t)
    print("saved mat", mat_fname)
    
print(mid_grid_x1.shape)

In [ ]:
# 27 interpolate AFTER predict, t=0

test_t0 = test_timesorted[np.where(np.abs(source_t) < 1e-8), :][0] # 2k

source_x1 = test_t0[:, 0]
source_x2 = test_t0[:, 1]
source_x3 = test_t0[:, 2]

# print(np.where(np.abs(source_t) < 1e-8))

# print(restored_vopt_xt.shape)
t0_vopt_xt = vopt_xt[np.where(np.abs(source_t) < 1e-8), :][0] # nx4
t0_v1 = t0_vopt_xt[:, 0]
t0_v2 = t0_vopt_xt[:, 1]
t0_v3 = t0_vopt_xt[:, 2]

print(t0_vopt_xt.shape)

t0_V1 = gd(
  (source_x1, source_x2, source_x3),
  t0_v1,
  (end_grid_x1, end_grid_x2, end_grid_x3),
  method='nearest')

t0_V2 = gd(
  (source_x1, source_x2, source_x3),
  t0_v2,
  (end_grid_x1, end_grid_x2, end_grid_x3),
  method='nearest')

t0_V3 = gd(
  (source_x1, source_x2, source_x3),
  t0_v3,
  (end_grid_x1, end_grid_x2, end_grid_x3),
  method='nearest')

print(t0_V1.shape)

save = True
if save:
    import scipy.io
    v = {
        "t0_V1" : t0_V1,
        "t0_V2" : t0_V2,
        "t0_V3" : t0_V3,
    }
    mat_fname = '%s/%s/notebook_post_predict_t0_%d_%d_v.mat' % (
        os.path.abspath("./"),
        id_prefix,
        vinterp_N,
        vinterp_T)
    scipy.io.savemat(mat_fname, v)
    print("saved mat", mat_fname)

In [ ]:
# 28 interpolate AFTER predict, middle

# middle = test_timesorted[(source_t < 5.0) & (source_t > 0.0), :] # 16k - 4k = 12k

middle = test_timesorted

source_x1 = middle[:, 0]
source_x2 = middle[:, 1]
source_x3 = middle[:, 2]
source_t_ = middle[:, 3]

# middle_vopt_xt = vopt_xt[(source_t < 5.0) & (source_t > 0.0), :]
middle_vopt_xt = vopt_xt

middle_v1 = middle_vopt_xt[:, 0]
middle_v2 = middle_vopt_xt[:, 1]
middle_v3 = middle_vopt_xt[:, 2]

print(middle_vopt_xt.shape)

mid_V1 = gd(
  (source_x1, source_x2, source_x3, source_t_),
  middle_v1,
  (mid_grid_x1, mid_grid_x2, mid_grid_x3, mid_grid_t),
  method='nearest')

mid_V2 = gd(
  (source_x1, source_x2, source_x3, source_t_),
  middle_v2,
  (mid_grid_x1, mid_grid_x2, mid_grid_x3, mid_grid_t),
  method='nearest')

mid_V3 = gd(
  (source_x1, source_x2, source_x3, source_t_),
  middle_v3,
  (mid_grid_x1, mid_grid_x2, mid_grid_x3, mid_grid_t),
  method='nearest')

print(mid_V1.shape)

In [ ]:
# 29 save mid to file

import scipy.io
v = {
    "mid_V1" : mid_V1,
    "mid_V2" : mid_V2,
    "mid_V3" : mid_V3,
}
mat_fname = '%s/%s/notebook_post_predict_mid_%d_%d_v.mat' % (
    os.path.abspath("./"),
    id_prefix,
    vinterp_N,
    vinterp_T)
print("abt to save")
scipy.io.savemat(mat_fname, v)
print("saved mat", mat_fname)

In [ ]:
print(np.max(mid_V3), np.min(mid_V3))

In [ ]:
# 30 interpolate AFTER predict, t=5

target_t = 5.0
target_thresh = 1e-8

ind = np.where(np.abs(source_t-target_t) < target_thresh)

test_t5 = test_timesorted[ind, :][0] # 2k

source_x1 = test_t5[:, 0]
source_x2 = test_t5[:, 1]
source_x3 = test_t5[:, 2]

# print(np.where(np.abs(source_t) < 1e-8))

# print(restored_vopt_xt.shape)
t5_vopt_xt = restored_vopt_xt[ind, :][0]
t5_v1 = t5_vopt_xt[:, 0]
t5_v2 = t5_vopt_xt[:, 1]
t5_v3 = t5_vopt_xt[:, 2]

print(t5_vopt_xt.shape)

t5_V1 = gd(
  (source_x1, source_x2, source_x3),
  t5_v1,
  (end_grid_x1, end_grid_x2, end_grid_x3),
  method='nearest')

t5_V2 = gd(
  (source_x1, source_x2, source_x3),
  t5_v2,
  (end_grid_x1, end_grid_x2, end_grid_x3),
  method='nearest')

t5_V3 = gd(
  (source_x1, source_x2, source_x3),
  t5_v3,
  (end_grid_x1, end_grid_x2, end_grid_x3),
  method='nearest')

print(t5_V1.shape)

save = True
if save:
    import scipy.io
    v = {
        "t5_V1" : t5_V1,
        "t5_V2" : t5_V2,
        "t5_V3" : t5_V3,
    }
    mat_fname = '%s/%s/post_predict_t5_%d_%d_v.mat' % (
        os.path.abspath("./"),
        id_prefix,
        vinterp_N,
        vinterp_T)
    scipy.io.savemat(mat_fname, v)
    print("saved mat", mat_fname)

In [ ]:
# 31 plot V1

fig1 = plt.figure()
ax1=fig1.gca(projection='3d')
sc1=ax1.scatter(source_x1, source_x2, source_x3, c=t5_v2, cmap=plt.hot())
plt.colorbar(sc1)
ax1.set_xlabel('x1')
ax1.set_ylabel('x2')
ax1.set_zlabel('x3')
ax1.set_title('source data t=%.3f' % (target_t))

#Plot interpolated values
fig2 = plt.figure()
ax2=fig2.gca(projection='3d')
sc2=ax2.scatter(end_grid_x1, end_grid_x2, end_grid_x3, c=t5_V2, cmap=plt.hot())
plt.colorbar(sc2)
ax2.set_xlabel('x1')
ax2.set_ylabel('x2')
ax2.set_zlabel('x3')
ax2.set_title('grid data t=%.3f' % (target_t))

plt.show()

In [ ]:
print(np.max(t0_V1)*j1)
print(np.max(t0_V2)*j2)
print(np.max(t0_V3)*j3)

print("#####")

print(np.min(t0_V1)*j1)
print(np.min(t0_V2)*j2)
print(np.min(t0_V3)*j3)

In [ ]:
# 32 sampling t0 vs

sampl = np.random.uniform(low=state_min, high=state_max, size=(1,3))

print(sampl)

closest_1 = [(np.abs(x_1_ - sampl[i, 0])).argmin() for i in range(sampl.shape[0])]
closest_2 = [(np.abs(x_2_ - sampl[i, 1])).argmin() for i in range(sampl.shape[0])]
closest_3 = [(np.abs(x_3_ - sampl[i, 2])).argmin() for i in range(sampl.shape[0])]

print(sampl[:, 0])
print(x_1_[closest_1])
print(closest_1)

v1s = t0_V1[closest_1, closest_2, closest_3]
v2s = t0_V2[closest_1, closest_2, closest_3]
v3s = t0_V3[closest_1, closest_2, closest_3]

print(v1s)

sampl = sampl[0, :]
print(sampl)

closest_1 = (np.abs(x_1_ - sampl[0])).argmin()
closest_2 = (np.abs(x_2_ - sampl[1])).argmin()
closest_3 = (np.abs(x_3_ - sampl[2])).argmin()

v1s = t0_V1[closest_1, closest_2, closest_3]
v2s = t0_V2[closest_1, closest_2, closest_3]
v3s = t0_V3[closest_1, closest_2, closest_3]

print(v1s)

In [ ]:
# 33 sampling middle vs

sampl = np.random.uniform(low=state_min, high=state_max, size=(1,4))
sampl[0,3] = np.random.uniform(low=0.1, high=4.9)

closest_1 = [(np.abs(x_1_ - sampl[i, 0])).argmin() for i in range(sampl.shape[0])]
closest_2 = [(np.abs(x_2_ - sampl[i, 1])).argmin() for i in range(sampl.shape[0])]
closest_3 = [(np.abs(x_3_ - sampl[i, 2])).argmin() for i in range(sampl.shape[0])]
closest_t = [(np.abs(t_ - sampl[i, 3])).argmin() for i in range(sampl.shape[0])]

print(sampl[:, 0])
print(x_1_[closest_1])
print(closest_1)

print(sampl[:, 3])
print(t_[closest_t])
print(closest_t)

v1s = mid_V1[closest_1, closest_2, closest_3, closest_t]
v2s = mid_V2[closest_1, closest_2, closest_3, closest_t]
v3s = mid_V3[closest_1, closest_2, closest_3, closest_t]

print(v1s)

In [ ]:
print(len(t0_V1.shape))

In [ ]:
# 34 interpolate BEFORE predict

# turn grid to table
table = np.vstack([grid_x1.reshape(-1), grid_x2.reshape(-1), grid_x3.reshape(-1), grid_t.reshape(-1)]).T
table = table[np.lexsort((table[:, 3], table[:,2], table[:,1], table[:,0]))]
print(table, table.shape)

table_tensor = tf.convert_to_tensor(table, dtype=tf.float32)
ind = np.lexsort((test[:,4],test[:,3])) # sorts by [3] (t) then by [4] (psi)
# sort AGAIN by output because a lot of samples @ t=0, t=5

with tf.GradientTape() as tape:
    tape.watch(table_tensor)

    # table_preds = model_restored.net(table_tensor)

    # you can take gradient of BOTH psi / rho w.r.t. input
    # but we only care about psi model.net[:, 0]
    table_psi = model_restored.net(table_tensor)[:, 0]

table_vopt_xt = tape.gradient(table_psi, table_tensor)
table_vopt_xt = table_vopt_xt.numpy()

print("")
print("table_vopt_xt")
print(table_vopt_xt)

table_vopt_xt_fname = "%s/%s/table_vopt_xt.dat" % (
    os.path.abspath("./"),
    id_prefix)
np.savetxt(table_vopt_xt_fname, table_vopt_xt)
print("saved table_vopt_xt")

In [ ]:
# 35 plot rho at t=5, t=0

target_t = 5.0

N = 50

test_timesorted = test[test[:, 3].argsort()]
timesorted = test_timesorted[:, 3]
test_ti = test_timesorted[np.where(np.abs(timesorted - target_t) < 1e-8), :][0] # 2k

ti_rho_opt = test_ti[:, 5]
ti_x1_x2_x3 = test_ti[:, 0:3]

####################################################################

d = 0.0
x1 = np.linspace(state_min - d, state_max + d, N)
x2 = np.linspace(state_min - d, state_max + d, N)
x3 = np.linspace(state_min - d, state_max + d, N)
X1, X2, X3 = np.meshgrid(x1,x2,x3,copy=False) # each is NxNxN

rho_opt = np.zeros((N,N,N))

closest_1 = [(np.abs(x1 - ti_x1_x2_x3[i, 0])).argmin() for i in range(ti_x1_x2_x3.shape[0])]
closest_2 = [(np.abs(x2 - ti_x1_x2_x3[i, 1])).argmin() for i in range(ti_x1_x2_x3.shape[0])]
closest_3 = [(np.abs(x3 - ti_x1_x2_x3[i, 2])).argmin() for i in range(ti_x1_x2_x3.shape[0])]

# some transposing going on in some reshape
# swapping closest_1/2 works well
rho_opt[closest_2, closest_1, closest_3] = ti_rho_opt

####################################################################

# RHO_OPT = gd(
#   (ti_x1_x2_x3[:, 0], ti_x1_x2_x3[:, 1], ti_x1_x2_x3[:, 2]),
#   ti_rho_opt,
#   (X1, X2, X3),
#   method='linear')

####################################################################

x1_marginal = np.array([
    np.trapz(
        np.array([
            np.trapz(rho_opt[j, i, :], x=x3) # x3 slices for one x2 => R
            for i in range(len(x2))]) # x3 slices across all x2 => Rn
        , x=x2) # x2 slice for one x1 => R
for j in range(len(x1))])

x2_marginal = np.array([
    np.trapz(
        np.array([
            np.trapz(rho_opt[i, j, :], x=x3) # x3 slices for one x1 => R
            for i in range(len(x1))]) # x3 slices across all x1 => Rn
        , x=x1) # x1 slice for one x2 => R
for j in range(len(x2))])

x3_marginal = np.array([
    np.trapz(
        np.array([
            np.trapz(rho_opt[i, :, j], x=x2) # x2 slices for one x1 => R
            for i in range(len(x1))]) # x2 slices across all x1 => Rn
        , x=x1) # x1 slice for one x3 => R
for j in range(len(x3))])

####################################################################

# normalize all the pdfs so area under curve ~= 1.0
x1_pdf_area = np.trapz(x1_marginal, x=x1)
x2_pdf_area = np.trapz(x2_marginal, x=x2)
x3_pdf_area = np.trapz(x3_marginal, x=x3)
print("prior to normalization: %.2f, %.2f, %.2f" % (
    x1_pdf_area,
    x2_pdf_area,
    x3_pdf_area))

x1_marginal /= x1_pdf_area
x2_marginal /= x2_pdf_area
x3_marginal /= x3_pdf_area

x1_pdf_area = np.trapz(x1_marginal, x=x1)
x2_pdf_area = np.trapz(x2_marginal, x=x2)
x3_pdf_area = np.trapz(x3_marginal, x=x3)
print("after to normalization: %.2f, %.2f, %.2f" % (
    x1_pdf_area,
    x2_pdf_area,
    x3_pdf_area))

In [ ]:
# 35 plot rho at t=5, t=0

fig = plt.figure(1)
ax1 = plt.subplot(131, frameon=False)
# ax1.set_aspect('equal')
ax1.grid()
ax1.set_title('x1 marginal')

ax2 = plt.subplot(132, frameon=False)
# ax2.set_aspect('equal')
ax2.grid()
ax2.set_title('x2 marginal')

# ax3 = plt.subplot(133, frameon=False)

ax3 = plt.subplot(133, frameon=False)
# ax3.set_aspect('equal')
ax3.grid()
ax3.set_title('x3 marginal')

colors="rgbymkc"

i = 0
t_e = 0
ax1.plot(x1,
    x1_marginal,
    colors[i % len(colors)],
    linewidth=1,
    label=t_e)
ax1.legend(loc='lower right')

ax2.plot(x2,
    x2_marginal,
    colors[i % len(colors)],
    linewidth=1,
    label=t_e)
ax2.legend(loc='lower right')

ax3.plot(x3,
    x3_marginal,
    colors[i % len(colors)],
    linewidth=1,
    label=t_e)
ax3.legend(loc='lower right')

fig.suptitle('t=%.2f' % (target_t), fontsize=16)

plt.show()